In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
num_classes = 100
batch_size = 100
epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train_onehot = to_categorical(y_train, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

# datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator()
generator_train = datagen.flow(x_train, y_train_onehot, batch_size=batch_size, seed=0)
generator_test = datagen.flow(x_test, y_test_onehot, batch_size=batch_size, seed=0)

In [3]:
def build_model():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',input_shape=x_train[0].shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

#     model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

In [4]:
retrain = False

if retrain:
    model = build_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit_generator(generator=generator_train,
                                        steps_per_epoch=x_train.shape[0] // batch_size,
                                        epochs=epochs,
                                        validation_data=generator_test,
                                        validation_steps=x_test.shape[0] // batch_size
                                 )
    model.save('models/VGG_CIFAR100.h5')                                     
else:
    model = load_model('models/VGG_CIFAR100.h5')

In [5]:
def acc(top_k, y_true):
    correct = 0
    for i in range(y_true.shape[0]):
        if y_true[i] in top_k[i]:
            correct += 1
    return correct/float(y_true.shape[0])

In [6]:
y_pred = model.predict(x_test)
print("Accuracy: %f" % (np.sum(np.argmax(y_pred, axis=1) == y_test.flatten()) / float(len(y_test))))

k = 5
y_pred_top_k = y_pred.argsort()[:,-k:]
print("Top %d accuracy: %f" % (k, acc(y_pred_top_k, y_test.flatten())))

Accuracy: 0.487600
Top 5 accuracy: 0.775600


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
__________

In [8]:
layer_name = 'max_pooling2d_5'
feat_extractor_model = Model(inputs= model.input, outputs = model.get_layer(layer_name).output)

In [9]:
train_feat = feat_extractor_model.predict(x_train)
print("train features:", train_feat.shape)
test_feat = feat_extractor_model.predict(x_test)
print("test features:", test_feat.shape)

train features: (50000, 1, 1, 512)
test features: (10000, 1, 1, 512)


In [10]:
np.save("vgg_16/train_x", train_feat)
np.save("vgg_16/train_y", y_train)
np.save("vgg_16/test_x", test_feat)
np.save("vgg_16/test_y", y_test)